In [92]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import os
dir_path = os.getcwd()


In [93]:
d_types = ['data', 'test']
data = dict()
for d_type in d_types:
        path = dir_path +"/"+"classification_data_HWK2/EMGaussian." + d_type 
        data[d_type] = pd.read_csv(path, sep=' ', header=None).values

In [94]:
K = 4

In [95]:
a = np.ones((K,K))/K

In [96]:
# We use the parameters found in HW2 - Q3 (We use our data)
pi = [0.20, 0.24, 0.25, 0.31]
centers = [[3.79, -3.64], [ 3.99, -3.64 ], [ -2.03, 4.16 ], [ -3.08, -3.56 ]]

sigma_0 = np.matrix([[ 0.87, 0.06 ], [ 0.06, 2.21]])
sigma_1 = np.matrix([[ 0.20, 0.22 ], [ 0.22, 10.40]])
sigma_2 = np.matrix([[ 2.92, 0.17 ], [ 0.17, 2.77]])
sigma_3 = np.matrix([[ 6.14, 5.94 ], [ 5.94, 6.07]])

sigmas = [sigma_0, sigma_1, sigma_2, sigma_3]

multivar = dict()
for k in range(K):
    m = scipy.stats.multivariate_normal(centers[k], sigmas[k])
    multivar[k] =m
    
# We also need to initiate A as it is not specified what is its value
# We follow Mr Chopin's advice to have strong weight on the diagonal
A =  1/10 * np.ones((4,4)) + 6/10 * np.identity(4)

Y_t_data = data['data']
T = len(Y_t_data)-1

In [97]:
def computeAlpha(z, Y_t_data, A,  t): 
    if t == 0:
        return pi[z]
    p_yt_zt = multivar[z].pdf(Y_t_data[t])
    alpha = 0
    for i in range(4):
        alpha += p_yt_zt * A[z, i] * computeAlpha(i, Y_t_data, A, t-1)
        
    return alpha

In [98]:
computeAlpha(2, Y_t_data, A, 8)

9.711462643780658e-22

In [99]:
def computeBeta(z, Y_t_data, A,  t): 
    if t == len(Y_t_data):
        return 1
    beta = 0
    for i in range(4):
        beta += A[z, i] * multivar[i].pdf(Y_t_data[t]) *  computeBeta(i, Y_t_data, A, t+1)
        
    return beta

In [100]:
computeBeta(2, Y_t_data, A, T-3)

2.7999202097087733e-08

In [101]:
# we note that values are very low, we thus need to use the log version to prevent numerical errors
log_A = np.log(A)
log_pi = np.log(pi)

# We will also use a matrix to store the already computed values of logAlpha and logBeta to prevent us from recomputing
# the whole recursivity at every step


In [108]:
def computeLogAlpha(z, Y_t_data, log_A, log_pi, memory_log_alpha, multivar,  t): 
    if t == 0:
        memory_log_alpha[z, t] = log_pi[z]
        return log_pi[z]
    p_yt_zt = multivar[z].pdf(Y_t_data[t])
    log_alpha = 0
    for i in range(4):
        log_alpha_i_tminus1 = 0 
        if memory_log_alpha[i, t-1] == 0:
            log_alpha_i_tminus1 = computeLogAlpha(i, Y_t_data, log_A, log_pi, memory_log_alpha,multivar,  t-1)
        else:
            log_alpha_i_tminus1 = memory_log_alpha[i, t-1]
        log_alpha += np.log(p_yt_zt) + log_A[z, i] + log_alpha_i_tminus1
    
    memory_log_alpha[z, t] = log_alpha  
    return log_alpha

In [109]:
memory_log_alpha = np.zeros((4, T + 1))
computeLogAlpha(2, Y_t_data, log_A, log_pi, memory_log_alpha,multivar,  8)

-6206742.28951718

In [110]:
def computeLogBeta(z, Y_t_data, log_A, log_pi, memory_log_beta, multivar,  t): 
    if t == T:  
        memory_log_beta[z, t] = 0
        return 0
    log_beta = 0
    for i in range(4):
        log_beta_i_tplus1 = 0 
        if memory_log_beta[i, t+1] == 0:
            log_beta_i_tplus1 = computeLogBeta(i, Y_t_data, log_A, log_pi, memory_log_beta,multivar, t+1)
        else:
            log_beta_i_tplus1 = memory_log_beta[i, t+1]
        
        log_beta += log_A[z, i] + np.log(multivar[i].pdf(Y_t_data[t])) +  log_beta_i_tplus1
      
    memory_log_beta[z, t] = log_beta  
    return log_beta

In [111]:
memory_log_beta = np.zeros((4, T + 1))
computeLogBeta(2, Y_t_data, log_A, log_pi, memory_log_beta,multivar, T-3)

-7304.243468618553

## 3. EM Algorithm

We will use the formulas found computed on the pdf file 
We will also initialize $\Pi_0$, $\mu_i$ and $\Sigma_i$ with the values found in the previous homework
For A, we will follow Mr Chopin's advice and we'll initiate our matrix with strong diagonal weights

In [133]:
def computeSmoothing(memory_log_alpha, memory_log_beta):
    p_zt_i = np.zeros((4, T))
    for t in range(T):
        memory_log_alpha_t = memory_log_alpha[:, t]
        memory_log_beta_t = memory_log_alpha[:, t]
        sum_memory_log_t = memory_log_alpha_t + memory_log_beta_t
        max_sum_log = np.min(-sum_memory_log_t)
        augmented_exp_sum_memory_log_t = np.exp(sum_memory_log_t + max_sum_log)
        for i in range(4):
            p_zt_i[i, t] = augmented_exp_sum_memory_log_t[i] / np.sum(augmented_exp_sum_memory_log_t)
            
    return p_zt_i

def computePzt1zt(memory_log_alpha, memory_log_beta, log_A, multivar, Y_t_data):
    p_zt1_j_zt_i = np.zeros((4,4, T))
    t = 0
    sum_memory_log_t = memory_log_alpha[:, t] + memory_log_beta[:, t]
    max_sum_memory_log_t = np.max(sum_memory_log_t)
    print("max_sum_memory_log_t", max_sum_memory_log_t)
    print("sum_memory_log_t", sum_memory_log_t)
    sum_memory_alphat_betat1 = memory_log_alpha[:, t] + memory_log_beta[:, t+1]
    for j in range(K):
        for i in range(K):
            print(max_sum_memory_log_t)
            tmpValue = memory_log_alpha[:, t][i] + memory_log_beta[:, t+1][j] + log_A[i,j] + np.log(multivar[j].pdf(Y_t_data[t+1]))
            below = max_sum_memory_log_t + np.log(np.sum(np.exp(sum_memory_log_t - max_sum_memory_log_t)))
            print("tmpValue:", tmpValue, "below:", below)
            value = np.exp(tmpValue + below)
            print(i, j, value)
            
            
            
#             sum_memory_alphat_betat1 = memory_log_alpha_t[i] + memory_log_beta_t1[j]
#             max_sum_log = np.max(sum_memory_alphat_betat1, max_sum_memory_log_t)
#             augmented_exp_sum_memory_log_t = np.exp(sum_memory_log_t - max_sum_log)   
#             ptji0 = np.exp(sum_memory_alphat_betat1 - max_sum_log)/np.sum(augmented_exp_sum_memory_log_t)
#             p_zt1_j_zt_i[j, i, t] = ptji0 * A[j, i] * 

In [136]:
def EM():
    Y_t_data = data['data']
    
    # init
    pi = [0.20, 0.24, 0.25, 0.31]
    centers = [[3.79, -3.64], [ 3.99, -3.64 ], [ -2.03, 4.16 ], [ -3.08, -3.56 ]]

    sigma_0 = np.matrix([[ 0.87, 0.06 ], [ 0.06, 2.21]])
    sigma_1 = np.matrix([[ 0.20, 0.22 ], [ 0.22, 10.40]])
    sigma_2 = np.matrix([[ 2.92, 0.17 ], [ 0.17, 2.77]])
    sigma_3 = np.matrix([[ 6.14, 5.94 ], [ 5.94, 6.07]])

    sigmas = [sigma_0, sigma_1, sigma_2, sigma_3]
    
    multivar = dict()
    for k in range(K):
        m = scipy.stats.multivariate_normal(centers[k], sigmas[k])
        multivar[k] =m
        A =  1/10 * np.ones((4,4)) + 6/10 * np.identity(4)
    
    
    p_z0_i = np.zeros((4,1))
    p_zt_i = np.zeros((4, T))
    p_zt1_j_zt_i = np.zeros((T, T))
    
    for j in range(1):
        # we use this memory to avoid unnecessary recomputation
        memory_log_alpha = np.zeros((4, T + 1))
        memory_log_beta = np.zeros((4, T + 1))
        # E step 
        log_A = np.log(A)
        log_pi = np.log(pi)
        print(log_pi)
        for i in range(4):
            # we fill memory_log_alpha and memory_log_beta
            _ = computeLogAlpha(i, Y_t_data, log_A, log_pi, memory_log_alpha,multivar, T)
            _ = computeLogBeta(i, Y_t_data, log_A, log_pi, memory_log_beta,multivar, 0)
        print(memory_log_alpha[:, 0])
        print(memory_log_beta[:, 0])
        p_zt_i = computeSmoothing(memory_log_alpha, memory_log_beta)
        p_zt1_j_zt_i = computePzt1zt(memory_log_alpha, memory_log_beta, log_A, multivar, Y_t_data)
        

In [137]:
EM()

[-1.60943791 -1.42711636 -1.38629436 -1.17118298]
[-1.60943791 -1.42711636 -1.38629436 -1.17118298]
[-3.10034595e+302 -3.10034595e+302 -3.10034595e+302 -3.10034595e+302]
max_sum_memory_log_t -3.1003459466837548e+302
sum_memory_log_t [-3.10034595e+302 -3.10034595e+302 -3.10034595e+302 -3.10034595e+302]
-3.1003459466837548e+302
tmpValue: -7.750864866709387e+301 below: -3.1003459466837548e+302
0 0 0.0
-3.1003459466837548e+302
tmpValue: -7.750864866709387e+301 below: -3.1003459466837548e+302
1 0 0.0
-3.1003459466837548e+302
tmpValue: -7.750864866709387e+301 below: -3.1003459466837548e+302
2 0 0.0
-3.1003459466837548e+302
tmpValue: -7.750864866709387e+301 below: -3.1003459466837548e+302
3 0 0.0
-3.1003459466837548e+302
tmpValue: -7.750864866709387e+301 below: -3.1003459466837548e+302
0 1 0.0
-3.1003459466837548e+302
tmpValue: -7.750864866709387e+301 below: -3.1003459466837548e+302
1 1 0.0
-3.1003459466837548e+302
tmpValue: -7.750864866709387e+301 below: -3.1003459466837548e+302
2 1 0.0
-3.1